In [1]:
from __future__ import division
import argparse

from PIL import Image
import numpy as np
import gym

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

2022-07-01 12:59:17.871265: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-01 12:59:17.871341: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

class AtariProcessor(Processor):
    """Class type Atari preprocessor based in keras-rl """
    def process_observation(self, observation):
        # (height, width, channel)
        assert observation.ndim == 3
        # resize image
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [3]:
def build_model(num_action):
    input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
    model = Sequential()

    model.add(Permute((2, 3, 1), input_shape=input_shape))
    model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(num_action))
    model.add(Activation('linear'))
    model.add(Flatten())

    return model

In [6]:
env = gym.make("Breakout-v0")
env.reset()
num_action = env.action_space.n
window = 4
model = build_model(num_action)
model.summary()
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.,
                              value_min=.1, value_test=.05, nb_steps=1000000)

dqn = DQNAgent(model=model, nb_actions=num_action, policy=policy,
               memory=memory, processor=processor, nb_steps_warmup=50000,
               gamma=.99, target_model_update=10000, train_interval=4,
               delta_clip=1.)

dqn.compile(Adam(lr=.00025), metrics=['mae'])
# training
dqn.fit(env,
        nb_steps=17500,
        log_interval=10000,
        visualize=False,
        verbose=2)

# save the final weights.
dqn.save_weights('policy.h5', overwrite=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 activation (Activation)     (None, 20, 20, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 activation_1 (Activation)   (None, 9, 9, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 activation_2 (Activation)   (None, 7, 7, 64)          0

/home/juand0145/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-07-01 15:08:52.889482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/juand0145/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-07-01 15:08:52.908551: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-01 15:08:52.933150: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-HTDB37B): /proc/driver/nvidia/version does not exist
2022-07-01 15:08:53.026848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Ne

Training for 17500 steps ...


/home/juand0145/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   224/17500: episode: 1, duration: 3.786s, episode steps: 224, steps per second:  59, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.594 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   403/17500: episode: 2, duration: 0.923s, episode steps: 179, steps per second: 194, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.436 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   629/17500: episode: 3, duration: 1.121s, episode steps: 226, steps per second: 202, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.451 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   972/17500: episode: 4, duration: 1.735s, episode steps: 343, steps per second: 198, episode reward:  3.000, mean reward:  0.009 [ 0.000,  1.000], mean action: 1.440 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1138/17500: episode: 5, duration: 0.950s, episode steps: 166, steps per second: 17

In [12]:
env = gym.make("Breakout-v0")
env.reset()
num_actions = env.action_space.n
model = build_model(num_action)  # deep conv net
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
dqn = DQNAgent(model=model, nb_actions=num_actions,
            processor=processor, memory=memory)
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

# load weights.
dqn.load_weights('policy.h5')

# evaluate algorithm for 10 episodes.
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...


/home/juand0145/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 1: reward: 0.000, steps: 10000
Episode 2: reward: 0.000, steps: 10000
Episode 3: reward: 0.000, steps: 10000
Episode 4: reward: 0.000, steps: 10000
Episode 5: reward: 0.000, steps: 10000
Episode 6: reward: 0.000, steps: 10000
Episode 7: reward: 0.000, steps: 10000
Episode 8: reward: 0.000, steps: 10000
Episode 9: reward: 0.000, steps: 10000
Episode 10: reward: 0.000, steps: 10000
